In [1]:
import cv2
import dlib
from scipy.spatial import distance as dist

In [4]:
def eyeAspectRatio(eye):
    a = dist.euclidean(eye[1], eye[5])
    b = dist.euclidean(eye[2], eye[4])
    c = dist.euclidean(eye[0], eye[3])
    ratio = (a+b)/(2.0*c)
    return ratio

In [2]:
def eyeDetection():
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye_tree_eyeglasses.xml')

    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Webcam not accessible.")
    else:
        while True:
            ret, frame = cap.read()
            if not ret:
                print("Error: Could not read frame.")
                break
        
        
            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            eyes = eye_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
        
            for (x, y, w, h) in eyes:
                cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
            
            cv2.imshow('Webcam Test', frame)
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        cap.release()
        cv2.destroyAllWindows()

In [3]:
eyeDetection()

In [ ]:
pip install --upgrade opencv-python dlib

In [2]:
import cv2
import dlib
import numpy as np  # Add this import
from scipy.spatial import distance

def calculate_EAR(eye):
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    ear_aspect_ratio = (A + B) / (2.0 * C)
    return ear_aspect_ratio

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open video source.")
    exit()

hog_face_detector = dlib.get_frontal_face_detector()
dlib_facelandmark = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

while True:
    ret, frame = cap.read()
    
    if not ret:
        print("Error: Failed to capture frame.")
        break

    # Ensure frame is in BGR format
    print(f"Frame type: {type(frame)}")
    print(f"Frame shape: {frame.shape}")
    
    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    print(f"Grayscale image type: {type(gray)}")
    print(f"Grayscale image shape: {gray.shape}")

    # Ensure the grayscale image is 8-bit
    if gray.dtype != 'uint8':
        print("Warning: Grayscale image is not 8-bit. Converting to uint8.")
        gray = cv2.convertScaleAbs(gray)
    
    # Explicitly ensure the image is in uint8
    gray = gray.astype('uint8')

    # Ensure dlib works with the image format by converting it to a numpy array
    gray = np.array(gray, dtype=np.uint8)
    
    # Face detection
    faces = hog_face_detector(gray)
    if len(faces) == 0:
        print("No faces detected.")
    
    for face in faces:
        face_landmarks = dlib_facelandmark(gray, face)
        leftEye = []
        rightEye = []

        for n in range(36, 42):
            x = face_landmarks.part(n).x
            y = face_landmarks.part(n).y
            leftEye.append((x, y))
            next_point = n + 1
            if n == 41:
                next_point = 36
            x2 = face_landmarks.part(next_point).x
            y2 = face_landmarks.part(next_point).y
            cv2.line(frame, (x, y), (x2, y2), (0, 255, 0), 1)

        for n in range(42, 48):
            x = face_landmarks.part(n).x
            y = face_landmarks.part(n).y
            rightEye.append((x, y))
            next_point = n + 1
            if n == 47:
                next_point = 42
            x2 = face_landmarks.part(next_point).x
            y2 = face_landmarks.part(next_point).y
            cv2.line(frame, (x, y), (x2, y2), (0, 255, 0), 1)

        left_ear = calculate_EAR(leftEye)
        right_ear = calculate_EAR(rightEye)

        EAR = (left_ear + right_ear) / 2
        EAR = round(EAR, 2)
        if EAR < 0.26:
            cv2.putText(frame, "DROWSY", (20, 100),
                        cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 255), 4)
            cv2.putText(frame, "Are you Sleepy?", (20, 400),
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 4)
            print("Drowsy")
        print(EAR)

    cv2.imshow("Are you Sleepy", frame)

    key = cv2.waitKey(1)
    if key == 27:  # Press 'ESC' to exit
        break

cap.release()
cv2.destroyAllWindows()



Frame type: <class 'numpy.ndarray'>
Frame shape: (480, 640, 3)
Grayscale image type: <class 'numpy.ndarray'>
Grayscale image shape: (480, 640)
No faces detected.
Frame type: <class 'numpy.ndarray'>
Frame shape: (480, 640, 3)
Grayscale image type: <class 'numpy.ndarray'>
Grayscale image shape: (480, 640)
No faces detected.
Frame type: <class 'numpy.ndarray'>
Frame shape: (480, 640, 3)
Grayscale image type: <class 'numpy.ndarray'>
Grayscale image shape: (480, 640)
No faces detected.
Frame type: <class 'numpy.ndarray'>
Frame shape: (480, 640, 3)
Grayscale image type: <class 'numpy.ndarray'>
Grayscale image shape: (480, 640)
No faces detected.
Frame type: <class 'numpy.ndarray'>
Frame shape: (480, 640, 3)
Grayscale image type: <class 'numpy.ndarray'>
Grayscale image shape: (480, 640)
No faces detected.
Frame type: <class 'numpy.ndarray'>
Frame shape: (480, 640, 3)
Grayscale image type: <class 'numpy.ndarray'>
Grayscale image shape: (480, 640)
No faces detected.
Frame type: <class 'numpy.nd

In [11]:
eyeStateDetection()

RuntimeError: Unsupported image type, must be 8bit gray or RGB image.

In [8]:
def detect_open_closed_eyes():
    # Load dlib's face detector and the shape predictor
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

    # Define eye aspect ratio threshold
    EAR_THRESHOLD = 0.2

    # Indices for the left and right eyes in the 68 facial landmarks model
    LEFT_EYE_IDX = slice(36, 42)  # Left eye landmarks
    RIGHT_EYE_IDX = slice(42, 48)  # Right eye landmarks

    # Open a connection to the webcam
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not access the webcam.")
        return

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Failed to capture frame from webcam.")
            break

        # Convert frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces in the frame
        faces = detector(gray)
        for face in faces:
            # Predict facial landmarks
            landmarks = predictor(gray, face)
            landmarks = [(landmarks.part(i).x, landmarks.part(i).y) for i in range(68)]

            # Extract left and right eye landmarks
            left_eye = landmarks[LEFT_EYE_IDX]
            right_eye = landmarks[RIGHT_EYE_IDX]

            # Calculate EAR for both eyes
            left_ear = eyeAspectRatio(left_eye)
            right_ear = eyeAspectRatio(right_eye)

            # Average EAR for both eyes
            avg_ear = (left_ear + right_ear) / 2.0

            # Determine if eyes are open or closed
            if avg_ear < EAR_THRESHOLD:
                eye_state = "Closed"
            else:
                eye_state = "Open"

            # Draw the eyes' state on the frame
            cv2.putText(frame, f"Eyes: {eye_state}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            # Draw rectangles around the eyes
            for (x, y) in left_eye + right_eye:
                cv2.circle(frame, (x, y), 2, (255, 0, 0), -1)

        # Display the frame
        cv2.imshow("Open/Closed Eye Detection", frame)

        # Exit on pressing 'q'
        key = cv2.waitKey(1)
        if key == ord('q'):
            break

    # Release the webcam and close all OpenCV windows
    cap.release()
    cv2.destroyAllWindows()

# Run the open/closed eye detection
detect_open_closed_eyes()


RuntimeError: Unsupported image type, must be 8bit gray or RGB image.

In [2]:
pip install scipy

  Using cached scipy-1.14.1-cp310-cp310-win_amd64.whl (44.8 MB)
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\User\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [5]:
import cv2
import dlib
import numpy as np
from scipy.spatial import distance

def calculate_EAR(eye):
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    ear_aspect_ratio = (A + B) / (2.0 * C)
    return ear_aspect_ratio

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open video source.")
    exit()

hog_face_detector = dlib.get_frontal_face_detector()
dlib_facelandmark = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Initialize variables to track EAR state over time
EAR_THRESHOLD_DROWSY = 0.26
EAR_THRESHOLD_SLEEPY = 0.20
EAR_THRESHOLD_SLEEPING = 0.18  # New threshold for sleeping state
frame_count = 0
drowsy_frames = 0
sleepy_frames = 0
sleeping_frames = 0
max_drowsy_frames = 10  # Threshold to consider as drowsy state
max_sleepy_frames = 15  # Threshold to consider as near-sleep state
max_sleeping_frames = 30  # Threshold to consider as sleeping state

while True:
    ret, frame = cap.read()
    
    if not ret:
        print("Error: Failed to capture frame.")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = gray.astype('uint8')
    gray = np.array(gray, dtype=np.uint8)
    
    faces = hog_face_detector(gray)
    if len(faces) == 0:
        print("No faces detected.")
    
    for face in faces:
        face_landmarks = dlib_facelandmark(gray, face)
        leftEye = []
        rightEye = []

        for n in range(36, 42):
            x = face_landmarks.part(n).x
            y = face_landmarks.part(n).y
            leftEye.append((x, y))
            next_point = n + 1
            if n == 41:
                next_point = 36
            x2 = face_landmarks.part(next_point).x
            y2 = face_landmarks.part(next_point).y
            cv2.line(frame, (x, y), (x2, y2), (0, 255, 0), 1)

        for n in range(42, 48):
            x = face_landmarks.part(n).x
            y = face_landmarks.part(n).y
            rightEye.append((x, y))
            next_point = n + 1
            if n == 47:
                next_point = 42
            x2 = face_landmarks.part(next_point).x
            y2 = face_landmarks.part(next_point).y
            cv2.line(frame, (x, y), (x2, y2), (0, 255, 0), 1)

        left_ear = calculate_EAR(leftEye)
        right_ear = calculate_EAR(rightEye)

        EAR = (left_ear + right_ear) / 2
        EAR = round(EAR, 2)

        if EAR < EAR_THRESHOLD_SLEEPING:
            sleeping_frames += 1
            if sleeping_frames >= max_sleeping_frames:
                cv2.putText(frame, "SLEEPING", (20, 100),
                            cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 255), 4)
                print("Sleeping")
        elif EAR < EAR_THRESHOLD_SLEEPY:
            sleepy_frames += 1
            if sleepy_frames >= max_sleepy_frames:
                cv2.putText(frame, "NEAR SLEEP", (20, 100),
                            cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 255), 4)
                print("Near Sleep")
        elif EAR < EAR_THRESHOLD_DROWSY:
            drowsy_frames += 1
            if drowsy_frames >= max_drowsy_frames:
                cv2.putText(frame, "DROWSY", (20, 100),
                            cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 255), 4)
                print("Drowsy")
        else:
            # Reset the frame counts when EAR is above thresholds
            drowsy_frames = 0
            sleepy_frames = 0
            sleeping_frames = 0

        print(f"EAR: {EAR}")

    cv2.imshow("Are you Sleepy", frame)

    key = cv2.waitKey(1)
    if key == 27:  # Press 'ESC' to exit
        break

cap.release()
cv2.destroyAllWindows()


EAR: 0.29
EAR: 0.29
EAR: 0.26
EAR: 0.29
EAR: 0.28
EAR: 0.31
EAR: 0.35
EAR: 0.33
EAR: 0.34
EAR: 0.31
EAR: 0.36
EAR: 0.32
EAR: 0.3
EAR: 0.31
EAR: 0.31
EAR: 0.3
EAR: 0.3
EAR: 0.21
EAR: 0.19
EAR: 0.21
EAR: 0.2
EAR: 0.19
EAR: 0.16
EAR: 0.14
EAR: 0.13
EAR: 0.15
EAR: 0.16
EAR: 0.16
EAR: 0.13
EAR: 0.16
EAR: 0.17
EAR: 0.18
EAR: 0.15
EAR: 0.17
EAR: 0.14
EAR: 0.23
EAR: 0.23
EAR: 0.21
EAR: 0.23
EAR: 0.24
EAR: 0.22
Drowsy
EAR: 0.23
Drowsy
EAR: 0.25
EAR: 0.17
Drowsy
EAR: 0.2
EAR: 0.19
EAR: 0.18
EAR: 0.19
EAR: 0.19
EAR: 0.18
EAR: 0.19
Drowsy
EAR: 0.22
Drowsy
EAR: 0.2
EAR: 0.17
EAR: 0.18
EAR: 0.18
Drowsy
EAR: 0.22
EAR: 0.15
EAR: 0.19
Drowsy
EAR: 0.2
Drowsy
EAR: 0.21
EAR: 0.19
Drowsy
EAR: 0.2
EAR: 0.19
Drowsy
EAR: 0.22
Near Sleep
EAR: 0.19
Near Sleep
EAR: 0.18
Drowsy
EAR: 0.2
Near Sleep
EAR: 0.19
Drowsy
EAR: 0.2
Near Sleep
EAR: 0.19
Near Sleep
EAR: 0.19
Drowsy
EAR: 0.2
Drowsy
EAR: 0.23
EAR: 0.27
EAR: 0.31
EAR: 0.33
EAR: 0.31
EAR: 0.31
EAR: 0.29
EAR: 0.31
EAR: 0.28
EAR: 0.29
EAR: 0.3
EAR: 0.11
EAR: 0.12